In [ ]:
import os
import numpy as np
import keras
from keras import layers, models, callbacks

In [ ]:
categories = [
    'aircraft carrier', 'airplane', 'ambulance', 'ant', 'anvil',
    'apple', 'axe', 'backpack', 'banana', 'baseball bat', 'baseball',
    'bat', 'bear', 'bed', 'bee', 'belt', 'bench', 'bird', 'book',
    'boomerang', 'bowtie', 'brain', 'bread', 'broom', 'bucket',
    'bulldozer', 'bus', 'The Eiffel Tower', 'The Mona Lisa'
]

data_path = './data/'

samples_per_category = 15000

X = []
y = []

for i, cat in enumerate(categories):
    file_path = os.path.join(data_path, f'full_numpy_bitmap_{cat}.npy')
    data = np.load(file_path)
    data = data[:samples_per_category]
    X.append(data)
    y.append(np.full(len(data), i))

X = np.concatenate(X).reshape(-1, 28, 28, 1).astype('float32') / 255.0
y = np.concatenate(y)

indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X, y = X[indices], y[indices]


print(f'X: {X.shape}, y: {y.shape}')

In [ ]:
model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),

    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPool2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPool2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(categories), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lr_reducer = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
early_stopper = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

model.save('draw-ai/models/temp_model.keras')